In [1]:
import pickle
import pandas as pd
from Embedder import Embedder, tfidfEmbedder
from OccupationPreprocessor import OccupationPreprocessor
import numpy as np

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Gradlab\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
with open('../first_dig_tfidf_clfs.pkl', 'rb') as f:
    clf1 = pickle.load(f)
    
with open('../second_third_fourth_dig_tfidf_clfs.pkl', 'rb') as f2:
    clf2 = pickle.load(f2)

In [3]:
TEST_FILE = 3

In [4]:
if TEST_FILE == 0:
    tfidf_test_set = OccupationPreprocessor.prepare_df(
        '../Data/ATP_gold_standard.xlsx',
        input_column='CURRENT_JOB_TITLE',
        code_column='NOC code by PATH',
        n_digits=4
    )
    
    NOCv5 = OccupationPreprocessor.prepare_df(
        '../Data/ATP_gold_standard.xlsx',
        input_column='NOC code by PATH',
        code_column='V5_NOC',
        n_digits=4
    )
    
    NOCv6 = OccupationPreprocessor.prepare_df(
        '../Data/ATP_gold_standard.xlsx',
        input_column='NOC code by PATH',
        code_column='V6_NOC',
        n_digits=4
    )
    
elif TEST_FILE == 1:
    tfidf_test_set = pd.DataFrame(
        pd.read_csv('../Data/tfidf_test_set.csv')
    )
    
# A-B test set with train data
elif TEST_FILE == 2:
    tfidf_test_set = pd.DataFrame(
        pd.read_csv('../Data/overlap_test_set_v4_acanoc_with_train_data.csv')
    )
    
# A-B test set no train data
elif TEST_FILE == 3:
    tfidf_test_set = pd.DataFrame(
        pd.read_csv('../Data/overlap_test_set_v4_acanoc_no_train_data.csv')
    )
    
tfidf_train_set = pd.DataFrame(
    pd.read_csv('../Data/tfidf_train_set.csv')
)

In [6]:
embedder = tfidfEmbedder()

Model loaded from ../vectorizer.joblib


In [7]:
sample_size = 500 if TEST_FILE == 0 else 100
sample_pipeline_df = tfidf_test_set.sample(sample_size)#, random_state=123)#.drop_duplicates()

### Metrics for alternative classifiers

In [12]:
from sklearn.metrics import accuracy_score, f1_score
def clean_noc_double_codes(row, column):
    if ',' in row[column]:
        row[column] = str(row[column].strip('\'').split(',')[1])
    return row

if TEST_FILE == 0:
    NOCv5 = NOCv5.apply(clean_noc_double_codes, axis =1 , args = ('input',))
    NOCv6 = NOCv6.apply(clean_noc_double_codes, axis =1 , args = ('input',))
    print('NOCv5 Ensemble metrics total. Accuracy:{}, f1:{}\n'.format(
        accuracy_score(NOCv5['input'].astype(int), NOCv5['code'].astype(int)), 
        f1_score(NOCv5['input'].astype(int), NOCv5['code'].astype(int), average = 'macro')
    ))
    print('NOCv6 Ensemble metrics total. Accuracy:{}, f1:{}\n'.format(
        accuracy_score(NOCv6['input'].astype(int), NOCv6['code'].astype(int)), 
        f1_score(NOCv6['input'].astype(int), NOCv6['code'].astype(int), average = 'macro')
    ))
elif TEST_FILE == 2 or TEST_FILE == 3:
    print(
        "Accuracy to Beat:",
        accuracy_score(
            sample_pipeline_df['code'], 
            sample_pipeline_df['v4_pred']
        ),
        "f1:", f1_score(
            sample_pipeline_df['code'], 
            sample_pipeline_df['v4_pred'],
            average='macro'
        )
    )

Accuracy to Beat: 0.39082209656372335 f1: 0.3582511168039559


### Check exact matches

In [13]:
tfidf_test_vectors = embedder.embed(sample_pipeline_df['input'])

In [14]:
tfidf_test_df = pd.DataFrame({
    'svm_pred':clf1['SVM'].predict(tfidf_test_vectors),
    'rf_pred':clf1['RF'].predict(tfidf_test_vectors),
    'knn_pred':clf1['KNN'].predict(tfidf_test_vectors),
    'code':sample_pipeline_df['code'].astype(str)
})

In [15]:
tfidf_test_df['exact_match'] = sample_pipeline_df['input'].apply(embedder.check_exact_match, args=(embedder.train_database,))

In [16]:
tfidf_test_df['p_all'] = tfidf_test_df.apply(tfidfEmbedder.ensemble_predict, axis = 1, args = (
    ['rf_pred','svm_pred','knn_pred'], 'svm_pred',
))

In [17]:
tfidf_test_df['vectors'] = tfidf_test_vectors.toarray().tolist()

In [18]:
# put in pipeline.py
def pipeline(row):
    np_array = np.array(row['vectors']).reshape(1, -1)
    p_1 = row['p_all']
    row['svm_pred_234'] = clf2[p_1]['SVM'].predict(np_array)[0]
    row['rf_pred_234'] = clf2[p_1]['RF'].predict(np_array)[0]
    row['knn_pred_234'] = clf2[p_1]['KNN'].predict(np_array)[0]
    return row

In [19]:
import time
start_time = time.time()
tfidf_test_df = tfidf_test_df.apply(pipeline, axis = 1)
elapsed = time.time()-start_time
print('time to predict on {0} samples: {1:.2f} seconds ({2:.3f} s/sample)'.format(sample_pipeline_df.shape[0],
                                                         elapsed, 
                                                        elapsed/sample_pipeline_df.shape[0])
     )

time to predict on 13794 samples: 3231.93 seconds (0.234 s/sample)


In [20]:
tfidf_test_df['p_all_234'] = tfidf_test_df.apply(tfidfEmbedder.ensemble_predict, axis = 1, args = (
    ['svm_pred_234','rf_pred_234','knn_pred_234'], 'knn_pred_234',
))

### performance w/o exact match

In [21]:
from sklearn.metrics import accuracy_score, f1_score

for classifier in ['knn','svm', 'rf']:
    print('{} acc:{}, f1-macro:{}'.format(classifier.upper(), 
                                    accuracy_score(
                                        tfidf_test_df['{}_pred_234'.format(classifier)].astype(int), 
                                        tfidf_test_df['code'].astype(int)
                                    ),
                                    f1_score(
                                        tfidf_test_df['{}_pred_234'.format(classifier)].astype(int),
                                        tfidf_test_df['code'].astype(int), average = 'macro')
                                   )
     )
print('Ensemble metrics total. Accuracy:{}, f1:{}\n'.format(
    accuracy_score(tfidf_test_df['p_all_234'].astype(int), tfidf_test_df['code'].astype(int)), 
    f1_score(tfidf_test_df['p_all_234'].astype(int), tfidf_test_df['code'].astype(int), average = 'macro')
))

KNN acc:0.42105263157894735, f1-macro:0.33485362754957915
SVM acc:0.4483833550819197, f1-macro:0.3789583334460654
RF acc:0.4391039582427142, f1-macro:0.3671925336906615
Ensemble metrics total. Accuracy:0.4572277801942874, f1:0.3765973041680976



### With exact match added

In [22]:
tfidf_test_df['Hybrid_pred_exact_match'] = tfidf_test_df.apply(
    lambda row : row['p_all_234'] if row['exact_match'] == -1 else row['exact_match'], axis = 1
)
print('Ensemble metrics after checking for exact match total. Accuracy:{}, f1:{}\n'.format(
    accuracy_score(tfidf_test_df['Hybrid_pred_exact_match'].astype(int), tfidf_test_df['code'].astype(int)), 
    f1_score(tfidf_test_df['Hybrid_pred_exact_match'].astype(int), tfidf_test_df['code'].astype(int), average = 'macro')
))

Ensemble metrics after checking for exact match total. Accuracy:0.4585326953748006, f1:0.37790825712765774



In [23]:
tfidf_test_df.drop(columns =['vectors'])

,svm_pred,rf_pred,knn_pred,code,exact_match,p_all,svm_pred_234,rf_pred_234,knn_pred_234,p_all_234,Hybrid_pred_exact_match
0,0,9,0,11,-1,0,11,11,414,11,11
1,0,9,0,11,-1,0,11,11,11,11,11
2,0,9,0,11,-1,0,12,821,432,432,432
3,0,9,0,11,-1,0,11,11,414,11,11
4,0,9,0,11,-1,0,11,11,11,11,11
...,...,...,...,...,...,...,...,...,...,...,...
13789,3,3,4,3011,-1,3,3011,3011,3011,3011,3011
13790,5,4,5,4021,-1,5,5254,5254,5241,5254,5254
13791,0,0,0,811,-1,0,631,621,311,311,311
13792,6,9,2,4423,-1,6,6541,6541,6541,6541,6541


In [42]:
import glob
from sklearn.metrics import f1_score
f1microsacanoc = []
f1macrosacanoc = []
f1microsme = []
f1macrosme = []
occupations = [] 
for csv in glob.glob('../output/*csv'):
    data_df = pd.read_csv('../output/'+csv)
    occupations += list(data_df['input'])
    f1microsacanoc.append(
        f1_score(data_df['acanoc_pred'],
        data_df['code'],
        average='micro')
    )
    f1microsme.append(
        f1_score(data_df['p_all_234'],
        data_df['code'],
        average='micro')
    )
    f1macrosacanoc.append(
        f1_score(data_df['acanoc_pred'],
        data_df['code'],
        average='macro')
    )
    f1macrosme.append(
        f1_score(data_df['p_all_234'],
        data_df['code'],
        average='macro')
    )

In [43]:
import scipy
scipy.stats.ttest_rel(f1microsme, f1microsacanoc, alternative='greater')

Ttest_relResult(statistic=55.18564031767717, pvalue=2.2591952492478045e-75)

In [44]:
scipy.stats.ttest_rel(f1macrosme, f1macrosacanoc, alternative='greater')

Ttest_relResult(statistic=15.37290622835621, pvalue=4.770444099046226e-28)

In [45]:
import plotly.figure_factory as ff

# Group data together
hist_data = [f1microsacanoc, f1microsme]

group_labels = ['Acanoc Density Estimation', 'TFIDF/D2V Ensemble Density Estimation']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=.2)
fig.add_trace(
    go.Histogram(
        x = f1microsme,
        name='TFIDF/D2V Ensemble Histogram'
    )
)
fig.add_trace(
    go.Histogram(
        x = f1microsacanoc,
        name='Acanoc Histogram'
    )
)
fig.update_layout(
    title='Distribution Plot of Micro F1 Scores',
    xaxis_title="Micro F1",
    yaxis_title="Frequency",
    paper_bgcolor='rgb(255, 255, 255)',
    plot_bgcolor='rgb(255, 255, 255)',
    font=dict(
        family= 'Times New Roman',
        size=14,
        color="black"
    ),
    xaxis_showgrid=True,
    yaxis_showgrid=True,
    legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.50
)
)
fig.update_xaxes(showline=True, linewidth=.5, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=.5, linecolor='black', mirror=True)
fig.update_traces(opacity=0.8)
fig.show()

In [46]:
# Group data together
hist_data = [f1macrosacanoc, f1macrosme]

group_labels = ['Acanoc Density Estimation', 'TFIDF/D2V Ensemble Density Estimation']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=.2)
fig.add_trace(
    go.Histogram(
        x = f1macrosme,
        name='TFIDF/D2V Ensemble Histogram'
    )
)
fig.add_trace(
    go.Histogram(
        x = f1macrosacanoc,
        name='Acanoc Histogram'
    )
)
fig.update_layout(
    title='Distribution Plot of Macro F1 Scores',
    xaxis_title="Macro F1",
    yaxis_title="Frequency",
    paper_bgcolor='rgb(255, 255, 255)',
    plot_bgcolor='rgb(255, 255, 255)',
    font=dict(
        family= 'Times New Roman',
        size=14,
        color="black"
    ),
    xaxis_showgrid=True,
    yaxis_showgrid=True,
    legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.50
)
)
fig.update_xaxes(showline=True, linewidth=.5, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
fig.update_traces(opacity=0.8)
fig.show()

In [74]:
scoring_plot = go.Figure()
x = np.arange(0, 11)
scoring_plot.add_trace(
    go.Scatter(x = x,
    y = x**0,
              mode ='lines',
              name='count scoring')
)


scoring_plot.add_trace(
    go.Scatter(x = x,
    y = [1/(xd+2) for xd in x],
              mode ='lines',
              name='hyperbolic scoring'
              )
)
scoring_plot.update_layout(
    title='Scoring Methods for D2V',
    xaxis_title="Index of Code in D2V Inference List",
    yaxis_title="Score",
    paper_bgcolor='rgb(255, 255, 255)',
    plot_bgcolor='rgb(255, 255, 255)',
    font=dict(
        family= 'Times New Roman',
        size=14,
        color="black"
    ),
    xaxis_showgrid=True,
    yaxis_showgrid=True,
    legend=dict(
    yanchor="top",
    y=0.90,
    xanchor="left",
    x=0.69
),
        width = 650
)
scoring_plot.update_xaxes(showline=True, linewidth=.5, linecolor='black', mirror=True)
scoring_plot.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
scoring_plot.show()